In [18]:
import os
import yfinance as yf
from google import genai
from google.genai import types
from dotenv import load_dotenv

In [19]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [20]:
def get_stock_data(ticker_symbol):
    """Fetches key metrics from Yahoo Finance."""
    stock = yf.Ticker(ticker_symbol)
    info = stock.info
    
    data_summary = {
        "name": info.get("longName"),
        "current_price": info.get("currentPrice"),
        "pe_ratio": info.get("forwardPE"),
        "market_cap": info.get("marketCap"),
        "52_week_high": info.get("fiftyTwoWeekHigh"),
        "52_week_low": info.get("fiftyTwoWeekLow"),
        "analyst_recommendation": info.get("recommendationKey"),
    }
    return data_summary

In [25]:
def get_ai_analysis(ticker, data):
    """Sends stock data to Gemini for analysis."""
    prompt = f"""
    You are a professional stock analyst. Analyze the following data for {ticker} ({data['name']}):
    - Current Price: ${data['current_price']}
    - Forward P/E Ratio: {data['pe_ratio']}
    - 52-Week Range: ${data['52_week_low']} - ${data['52_week_high']}
    - Market Cap: {data['market_cap']}
    - General Analyst Consensus: {data['analyst_recommendation']}

    Task:
    1. Determine if the stock is a 'Buy', 'Hold', or 'Sell'.
    2. Provide a 3-sentence justification focusing on the stock's recent performance, P/E ratio, and price position.
    3. State that this is NOT financial advice.
    """
    
    response = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        contents=prompt
    )
    return response.text

In [26]:
user_ticker = input("Enter a stock ticker (e.g., AAPL): ").upper()
print(f"--- Fetching data for {user_ticker} ---")

try:
    stock_data = get_stock_data(user_ticker)
    analysis = get_ai_analysis(user_ticker, stock_data)
    
    print("\n--- Gemini's Analysis ---")
    print(analysis)
except Exception as e:
    print(f"Error: Could not retrieve data for {user_ticker}. Check the ticker symbol.", e)

--- Fetching data for AAPL ---

--- Gemini's Analysis ---
Here's an analysis of AAPL based on the provided data:

**1. Recommendation:** **Buy**

**2. Justification:** AAPL is trading near the higher end of its 52-week range, reflecting strong recent performance and investor confidence. Its forward P/E ratio of approximately 29 suggests investors are willing to pay a premium for its future earnings, a common characteristic of growth companies. Given the general analyst consensus of "buy" and the stock's demonstrated ability to perform within its historical price bounds, it presents a potentially attractive investment opportunity.

**3. Disclaimer:** This analysis is for informational purposes only and does not constitute financial advice.
